In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninst

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import open3d as o3d
from tqdm import tqdm
import os
import glob
import cv2
from pathlib import Path

class PointCloudRenderer(nn.Module):
    def __init__(self, img_height=394, img_width=526, hidden_dim=32):
        super().__init__()
        self.img_height = img_height
        self.img_width = img_width

        self.feature_net = nn.Sequential(
            nn.Linear(6, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.renderer = nn.Sequential(
            nn.Linear(hidden_dim + 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3),
            nn.Sigmoid()
        )

    @staticmethod
    def project_points(points, camera_pos, camera_dir, img_width, img_height):
        focal_length = 800
        principal_point = np.array([img_width/2, img_height/2])

        camera_up = np.array([0, -1, 0])
        camera_right = np.cross(camera_dir, camera_up)
        camera_up = np.cross(camera_right, camera_dir)

        R = np.stack([
            camera_right / np.linalg.norm(camera_right),
            camera_up / np.linalg.norm(camera_up),
            camera_dir / np.linalg.norm(camera_dir)
        ], axis=1)

        points_cam = (points - camera_pos) @ R
        valid_mask = points_cam[:, 2] > 0
        points_cam = points_cam[valid_mask]

        if len(points_cam) == 0:
            return None, None, valid_mask

        points_2d = points_cam[:, :2] / points_cam[:, 2:3] * focal_length + principal_point
        return points_2d, points_cam[:, 2], valid_mask

def render_frame(points, colors, camera, img_width, img_height):
    image_coords, depths, valid_mask = PointCloudRenderer.project_points(
        points, camera['position'], camera['direction'], img_width, img_height
    )

    if image_coords is None:
        return np.zeros((img_height, img_width, 3))

    frame = np.zeros((img_height, img_width, 3))
    depth_buffer = np.full((img_height, img_width), np.inf)

    image_coords = np.clip(image_coords, 0, [img_width-1, img_height-1]).astype(int)

    for idx in range(len(image_coords)):
        x, y = image_coords[idx]
        if 0 <= y < img_height and 0 <= x < img_width:
            if depths[idx] < depth_buffer[y, x]:
                frame[y, x] = colors[valid_mask][idx]
                depth_buffer[y, x] = depths[idx]

    frame = cv2.GaussianBlur(frame, (3, 3), 0)
    return frame

def load_point_cloud(file_path):
    pcd = o3d.io.read_point_cloud(file_path)

    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)

    if len(points) == 0:
        raise ValueError(f"No points found in {file_path}")

    center = np.mean(points, axis=0)
    points = points - center

    points = points * np.array([1, -1, 1])

    scale = np.max(np.abs(points))
    points = points / (scale * 2)

    return points, colors

def generate_camera_movement(n_frames):
    radius = 2.0
    height = 0.2
    orbit_angle = 60

    orbit_rad = np.radians(orbit_angle)

    t = np.linspace(0, 2 * np.pi, n_frames)
    angles = -(orbit_rad/2) * np.cos(t)

    cameras = []

    for angle in angles:
        x = radius * np.sin(angle)
        z = radius * np.cos(angle)
        pos = np.array([x, height, z])

        direction = -pos / np.linalg.norm(pos)

        cameras.append({
            'position': pos,
            'direction': direction
        })

    return cameras

def create_3d_video(input_folder, output_path, img_width=480, img_height=270):
    ply_files = sorted(glob.glob(os.path.join(input_folder, '*.ply')))
    if not ply_files:
        raise ValueError(f"No PLY files found in {input_folder}")

    print("Loading first point cloud to check coordinates...")
    points, _ = load_point_cloud(ply_files[0])
    print(f"\nPoint cloud bounds after processing:")
    print(f"X: [{points[:,0].min():.2f}, {points[:,0].max():.2f}]")
    print(f"Y: [{points[:,1].min():.2f}, {points[:,1].max():.2f}]")
    print(f"Z: [{points[:,2].min():.2f}, {points[:,2].max():.2f}]")

    cameras = generate_camera_movement(len(ply_files))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(
        output_path,
        fourcc,
        30,
        (img_width, img_height)
    )

    for ply_file, camera in tqdm(zip(ply_files, cameras), total=len(ply_files), desc="Processing frames"):
        try:
            points, colors = load_point_cloud(ply_file)
            frame = render_frame(points, colors, camera, img_width, img_height)
            frame_rgb = (np.clip(frame, 0, 1) * 255).astype(np.uint8)
            video_writer.write(cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))

        except Exception as e:
            print(f"Error processing {ply_file}: {e}")
            continue

    video_writer.release()
    print(f"Video saved as: {output_path}")

if __name__ == "__main__":
    input_folder = '/content/drive/MyDrive/point_clouds'
    output_path = 'output_video.mp4'
    create_3d_video(input_folder, output_path)

Loading first point cloud to check coordinates...
Point cloud bounds before centering:
X: [-111.78, 63.36]
Y: [-111.77, 14.33]
Z: [2.00, 255.00]

Point cloud bounds after processing:
X: [-0.33, 0.26]
Y: [-0.10, 0.32]
Z: [-0.35, 0.50]


Processing frames:   0%|          | 0/58 [00:00<?, ?it/s]

Point cloud bounds before centering:
X: [-111.78, 63.36]
Y: [-111.77, 14.33]
Z: [2.00, 255.00]


Processing frames:   2%|▏         | 1/58 [01:39<1:34:21, 99.33s/it]

Point cloud bounds before centering:
X: [-112.70, 61.66]
Y: [-113.14, 14.33]
Z: [2.00, 255.00]


...

Point cloud bounds before centering:
X: [-108.63, 52.89]
Y: [-90.78, 13.20]
Z: [3.00, 255.00]


Processing frames:  98%|█████████▊| 57/58 [1:36:21<01:08, 68.85s/it]

Point cloud bounds before centering:
X: [-112.18, 55.07]
Y: [-93.06, 13.82]
Z: [3.00, 255.00]


Processing frames: 100%|██████████| 58/58 [1:37:39<00:00, 101.03s/it]

Video saved as: output_video.mp4
